# 命令模式

命令模式(Command Pattern): 将一个请求封装为一个对象，从而让我们可用不同的请求对客户进行参数化；对请求排队或者记录请求日志，以及支持可撤销的操作。其别名为动作(Action)模式或事务(Transaction)模式

命令模式的核心在于引入了命令类，通过命令类来降低发送者和接收者的耦合度, 发送者与接收者之间没有直接引用关系，发送请求的对象只需要知道如何发送请求，而不必知道如何完成请求;
命令模式的本质是对请求进行封装，一个请求对应于一个命令，将发出命令的责任和执行命令的责任分割开;
一个请求发送者可以对应多个命令对象, 从而可以实现**命令队列**;

**宏命令:**

宏命令(Macro Command)又称为组合命令，它是组合模式和命令模式联用的产物。宏命令是一个具体命令类，它拥有一个集合属性，在该集合中包含了对其他命令对象的引用。通常宏命令不直接与请求接收者交互，而是通过它的成员来调用接收者的方法。当调用宏命令的`execute()`方法时，将递归调用它所包含的每个成员命令的`execute()`方法，一个宏命令的成员可以是简单命令，还可以继续是宏命令。执行一个宏命令将触发多个具体命令的执行，从而实现对命令的批处理.

包含角色:
- Command: 一般是抽象类, 声明执行请求的`execute()`等方法
- ConcreteCommand: 对应具体的接受者, 在`execute()`中调用接受者相关操作
- Invoker: 请求发送者, 通过命令对象执行请求
- Receiver: 接受者执行请求相关操作

结构图:

![](https://github.com/luog1992/fs/blob/master/2017110201.png?raw=true)

## 示例

In [11]:
# invoker
class Button(object):
    def __init__(self, name, command=None):
        self.name = name
        self.command = command

    def on_click(self):
        self.command.execute()


# abstract command
class Command(object):
    def __init__(self, name, receive=None):
        self.name = name
        self.receive = receive

    def execute(self):
        raise NotImplementedError


# concrete command 1 without a receiver
class CloseWindow(Command):
    def execute(self):
        print 'Closing window'


# concrete command 2 with a receive
class ImageZoomIn(Command):
    def execute(self):
        print 'Zoom in the image...'
        self.receive.zoom_in()


# a receive
class Image(object):
    def zoom_in(self, percent=75):
        print 'Image zoom in to %s%%' % percent

In [5]:
close_btn = Button('close_btn', CloseWindow('close_cmd'))
close_btn.on_click()

Closing window


In [10]:
zoomin_btn = Button('zoomin_btn', ImageZoomIn('img_zoomin_cmd', Image()))
zoomin_btn.on_click()

Zoom in the image...
Image zoom in to 75%


## 通过命令模式实现撤销操作

In [17]:
# receive
class Adder(object):
    def __init__(self):
        self.n = 0

    def add(self, val):
        self.n += val
        return self.n


# abstract command
class Command(object):
    def execute(self, val):
        raise NotImplementedError

    def undo(self):
        raise NotImplementedError


# concrete add command
class AddCommand(Command):
    def __init__(self, adder):
        self.adder = adder
        self._histories = []

    def execute(self, val):
        self._histories.append(val)
        return self.adder.add(val)

    def undo(self):
        if not self._histories:
            raise ValueError('No more undo steps')
        return self.adder.add(-self._histories.pop())


# invoker
class Calculator(object):
    def __init__(self, add_command):
        self._add_command = add_command

    def compute(self, val):
        r = self._add_command.execute(val)
        print '调用:', val, r

    def undo(self):
        r = self._add_command.undo()
        print '撤销:', r

In [19]:
adder = Adder()
command = AddCommand(adder)
c = Calculator(command)

c.compute(10)
c.compute(12)
c.undo()
c.compute(5)
c.undo()
c.undo()

调用: 10 10
调用: 12 22
撤销: 10
调用: 5 15
撤销: 10
撤销: 0


In [20]:
c.undo()

ValueError: No more undo steps